In [1]:
import * as hardhat from 'hardhat'
import '@nomiclabs/hardhat-ethers'
import { ethers } from 'hardhat'
import {smock} from '@defi-wonderland/smock'

import * as bn from './utils/bn'
import * as evm from './utils/evm'
import * as wallet from './utils/wallet'

import {getMainnetSdk} from '@dethcrypto/eth-sdk-client';

In [2]:
// FORK
let blockNumber = 16000000

await evm.reset({
    jsonRpcUrl: process.env['MAINNET_RPC'],
    blockNumber
})

const provider = ethers.provider

In [3]:
const signer = await ethers.getSigner()
const signerAddress = await signer.getAddress()

In [4]:
await wallet.fund(signer.address, bn.toUnit(100))

In [5]:
const gameAbi = require('./out/ButtPlugWars.sol/ButtPlugWars.json')
const gameFactory = await ethers.getContractFactory(gameAbi.abi, gameAbi.bytecode.object, signer)
const game = await gameFactory.deploy({gasLimit: 10e6})

In [6]:
const FIVEOUTOFNINE_WHALE = '0xC5233C3b46C83ADEE1039D340094173f0f7c1EcF'
const whale = await wallet.impersonate(FIVEOUTOFNINE_WHALE)
const KEEPER_ADDRESS = '0x9429cd74a3984396f3117d51cde46ea8e0e21487'
const keeper = await wallet.impersonate(KEEPER_ADDRESS)

In [7]:
const {chess, sudoPool} = getMainnetSdk(signer)
const officialSudoPool = sudoPool.attach(await game.SUDOSWAP_POOL())

In [8]:
await evm.advanceTimeAndBlock(86400*10)
await game.startEvent();
'game-start'

game-start


In [9]:
await chess.connect(whale).transferFrom(whale._address, signer.address, 180)
await chess.connect(whale).transferFrom(whale._address, signer.address, 181)
await chess.setApprovalForAll(game.address, true)
await game.mintPlayerBadge(180, 0, {value: bn.toUnit(1)})
await game.mintPlayerBadge(181, 1, {value: bn.toUnit(1)})
'player mint'

player mint


In [10]:
await evm.advanceTimeAndBlock(14 * 86400)
await game.pushLiquidity()
'push-liquidity'

push-liquidity


In [11]:
await game.tokenURI(1)

{"name" : "Player", "description" : "ButtPlug Wars player badge", "image_data" : "", "attributes" : [{"trait_type" : "team", "value" : "ZERO"}, {"trait_type" : "weight", "value" : 1000}, {"trait_type" : "score", "value" : 0}, {"trait_type" : "vote", "value" : "0x0000000000000000000000000000000000000000"}, {"trait_type" : "bonded_token", "value" : "180"}]}


In [12]:
const buttPlugAbi = require('./out/IGame.sol/IButtPlug.json')
const fakeButtPlug = await smock.fake(buttPlugAbi.abi)

In [13]:
await game.getBadgeId(2)

BigNumber { _hex: '0x0800000000000002', _isBigNumber: true }


In [14]:
// await game.mintButtPlugBadge(fakeButtPlug.address)
await game.voteButtPlug(fakeButtPlug.address, 1);
await game.voteButtPlug(fakeButtPlug.address, '0x0800000000000002');
'vote fake buttplug'

vote fake buttplug


In [15]:
await chess.connect(whale).transferFrom(whale._address, keeper._address, 182)
'add nft to keeper'

add nft to keeper


In [16]:
await chess.board()

BigNumber {
  _hex: '0x03256230011111100000000000000000099999900bcdecb000000001',
  _isBigNumber: true
}


In [17]:
await evm.advanceTimeAndBlock(5*86400)
fakeButtPlug.readMove.returns((10 << 6) | 25)
await game.connect(keeper).executeMove()
console.log((await officialSudoPool.getBuyNFTQuote(1)).inputAmount.toString())

await evm.advanceTimeAndBlock(5*86400)
fakeButtPlug.readMove.returns((13 << 6) | 30)
await game.connect(keeper).executeMove()
console.log((await officialSudoPool.getBuyNFTQuote(1)).inputAmount.toString())

await evm.advanceTimeAndBlock(5*86400)
fakeButtPlug.readMove.returns((20 << 6) | 28)
await game.connect(keeper).executeMove()
console.log((await officialSudoPool.getBuyNFTQuote(1)).inputAmount.toString())

// reverts
await evm.advanceTimeAndBlock(5*86400)
fakeButtPlug.readMove.returns((12 << 6) | 28)
await game.connect(keeper).executeMove()
console.log((await officialSudoPool.getBuyNFTQuote(1)).inputAmount.toString())

await evm.advanceTimeAndBlock(5*86400)
fakeButtPlug.readMove.returns((28 << 6) | 42)
await game.connect(keeper).executeMove()
console.log((await officialSudoPool.getBuyNFTQuote(1)).inputAmount.toString())

59295000000000000
29647500000000000
19764999999999999
19764999999999999
19764999999999999


In [18]:
// can buy 3 minted tokens
await officialSudoPool.swapTokenForAnyNFTs(1, bn.toUnit(1), signer.address, false, signer.address, {value: bn.toUnit(1)})
console.log((await officialSudoPool.getBuyNFTQuote(1)).inputAmount.toString())
await officialSudoPool.swapTokenForAnyNFTs(1, bn.toUnit(1), signer.address, false, signer.address, {value: bn.toUnit(1)})
console.log((await officialSudoPool.getBuyNFTQuote(1)).inputAmount.toString())
await officialSudoPool.swapTokenForAnyNFTs(1, bn.toUnit(1), signer.address, false, signer.address, {value: bn.toUnit(1)})
console.log((await officialSudoPool.getBuyNFTQuote(1)).inputAmount.toString())

39529999999999999
118589999999999998
0


In [19]:
await game.tokenURI(0)

{"name" : "Scoreboard", "description" : "ButtPlug Wars Scoreboard 
0(-2) - 0(-1) 
```
6 ♜ ♝ ♛ ♚ ♝ ♜
5 ♟ · ♟ · ♟ ♟
4 · · · ♟ · ·
3 ♟ · ♙ · · ♘
2 ♙ ♙ · ♙ ♙ ♙
1 ♖ · ♕ ♔ · ♖
  a b c d e f
```", "image_data" : "", "attributes" : [{"trait_type" : "game-score", "value" : "0(-2) - 0(-1)"}, {"trait_type" : "weight", "value" : 2000}]}


In [20]:
JSON.parse(await game.tokenURI(1))

{
  name: 'Player',
  description: 'ButtPlug Wars player badge',
  image_data: '',
  attributes: [
    { trait_type: 'team', value: 'ZERO' },
    { trait_type: 'weight', value: 1000 },
    { trait_type: 'score', value: -2000 },
    {
      trait_type: 'vote',
      value: '0x948255ece598e31492a2412d3d73e32b8e9845b8'
    },
    { trait_type: 'bonded_token', value: '180' }
  ]
}


In [21]:
await game.tokenURI('0x0800000000000002')

{"name" : "Player", "description" : "ButtPlug Wars player badge", "image_data" : "", "attributes" : [{"trait_type" : "team", "value" : "ONE"}, {"trait_type" : "weight", "value" : 1000}, {"trait_type" : "score", "value" : -1000}, {"trait_type" : "vote", "value" : "0x948255ece598e31492a2412d3d73e32b8e9845b8"}, {"trait_type" : "bonded_token", "value" : "181"}]}


In [22]:
await chess.board()

BigNumber {
  _hex: '0x032562300101011000001000010900c0099099900b0de0b000000001',
  _isBigNumber: true
}


In [23]:
const chessAbi = require('./eth-sdk/abis/mainnet/chess.json')
const fakeChess = await smock.fake(chessAbi, {address: chess.address})

In [24]:
const NEW_BOARD = '0x03256230011111100000000000000000099999900bcdecb000000001'
const WHITE_CAP = '0x03256230011111100000000000000000099909900bcdecb000000001'
const BLACK_CAP = '0x03256230011011100000000000000000099999900bcdecb000000001'
const BOTH_CAPS = '0x03256230011011100000000000000000099909900bcdecb000000001'

In [39]:
fakeChess.board.reset();
fakeChess.balanceOf.returns(9);
fakeChess.board.returnsAtCall(0, NEW_BOARD)
fakeChess.board.returnsAtCall(1, NEW_BOARD)
await evm.advanceTimeAndBlock(5*86400)
await game.connect(keeper).executeMove()

{
  hash: '0x7505ae760e1c276b12ccb747059e4df42ba8b2dc503ba9cc6f4052248d6cd20d',
  type: 2,
  accessList: [],
  blockHash: '0xa794f5a0a80f6b97585dc8f54b21e6317d806bcfde44afa515f59e6832486891',
  blockNumber: 16000046,
  transactionIndex: 0,
  confirmations: 1,
  from: '0x9429cd74A3984396f3117d51cde46ea8e0e21487',
  gasPrice: BigNumber { _hex: '0x3d9ebeeb', _isBigNumber: true },
  maxPriorityFeePerGas: BigNumber { _hex: '0x3b9aca00', _isBigNumber: true },
  maxFeePerGas: BigNumber { _hex: '0x3fa2b3d6', _isBigNumber: true },
  gasLimit: BigNumber { _hex: '0x01bad458', _isBigNumber: true },
  to: '0x3D63c50AD04DD5aE394CAB562b7691DD5de7CF6f',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  nonce: 515,
  data: '0x30e6d8b2',
  r: '0x0000000000000000000000000000000000000000000000000000000000000001',
  s: '0x0000000000000000000000000000000000000000000000000000000000000002',
  v: 1,
  creates: null,
  chainId: 31337,
  wait: [Function (anonymous)]
}


In [40]:
await game.tokenURI(0)

{"name" : "Scoreboard", "description" : "ButtPlug Wars Scoreboard 
4(0) - 5(0) 
```
6 · · · · · ·
5 · · · · · ·
4 · · · · · ·
3 · · · · · ·
2 · · · · · ·
1 · · · · · ·
  a b c d e f
```", "image_data" : "", "attributes" : [{"trait_type" : "game-score", "value" : "4(0) - 5(0)"}, {"trait_type" : "weight", "value" : 2000}]}


In [41]:
JSON.parse(await game.tokenURI(1))

{
  name: 'Player',
  description: 'ButtPlug Wars player badge',
  image_data: '',
  attributes: [
    { trait_type: 'team', value: 'ZERO' },
    { trait_type: 'weight', value: 1000 },
    { trait_type: 'score', value: 10000 },
    {
      trait_type: 'vote',
      value: '0x948255ece598e31492a2412d3d73e32b8e9845b8'
    },
    { trait_type: 'bonded_token', value: '180' }
  ]
}


In [42]:
JSON.parse(await game.tokenURI('0x0800000000000002'))

{
  name: 'Player',
  description: 'ButtPlug Wars player badge',
  image_data: '',
  attributes: [
    { trait_type: 'team', value: 'ONE' },
    { trait_type: 'weight', value: 1000 },
    { trait_type: 'score', value: 14000 },
    {
      trait_type: 'vote',
      value: '0x948255ece598e31492a2412d3d73e32b8e9845b8'
    },
    { trait_type: 'bonded_token', value: '181' }
  ]
}


In [43]:
await game.unbondLiquidity()

{
  hash: '0x71236f979be20baad3f9e8857becd1d91b8faeb2eab3da3c566095df7d55d7f0',
  type: 2,
  accessList: [],
  blockHash: '0xe10b0358ccf8eb38278e88abf5a23407bc995ef6b92fee47c786d8934432032b',
  blockNumber: 16000047,
  transactionIndex: 0,
  confirmations: 1,
  from: '0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266',
  gasPrice: BigNumber { _hex: '0x3d5f4659', _isBigNumber: true },
  maxPriorityFeePerGas: BigNumber { _hex: '0x3b9aca00', _isBigNumber: true },
  maxFeePerGas: BigNumber { _hex: '0x3f23c2b2', _isBigNumber: true },
  gasLimit: BigNumber { _hex: '0x01bad458', _isBigNumber: true },
  to: '0x3D63c50AD04DD5aE394CAB562b7691DD5de7CF6f',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  nonce: 362,
  data: '0x7152d519',
  r: '0x4ad845cd49b0ee0b1d08236a81cfa6991bcac693758db4056a41cc4c4f14bc61',
  s: '0x17b9139e4702ac4607213f086f99454867b8abceb743f9301785dfd0098f2042',
  v: 1,
  creates: null,
  chainId: 31337,
  wait: [Function (anonymous)]
}


In [44]:
await game.claimPrize('0x0800000000000002')

{
  hash: '0xc89b654c9f7724d46c5d8954879380adb639edf31cf1d98af2d763f410794022',
  type: 2,
  accessList: [],
  blockHash: '0x3921bc49fcb7fff11c138175bd5e1edb178ec159eb04c27321dbd3f3ab870b67',
  blockNumber: 16000048,
  transactionIndex: 0,
  confirmations: 1,
  from: '0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266',
  gasPrice: BigNumber { _hex: '0x3d272f4e', _isBigNumber: true },
  maxPriorityFeePerGas: BigNumber { _hex: '0x3b9aca00', _isBigNumber: true },
  maxFeePerGas: BigNumber { _hex: '0x3eb3949c', _isBigNumber: true },
  gasLimit: BigNumber { _hex: '0x01bad558', _isBigNumber: true },
  to: '0x3D63c50AD04DD5aE394CAB562b7691DD5de7CF6f',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  nonce: 363,
  data: '0xd70981540000000000000000000000000000000000000000000000000800000000000002',
  r: '0xe32c68dfbf7af42fad3e34dba9a51ad642197ab8e06d84fcc5baf86eb088bc83',
  s: '0x6756fdca27aeb94596a1c2f5828be813073b84e56ab6b92a8ab91b21164dc00c',
  v: 1,
  creates: null,
  chainId: 31337,
  wa

In [45]:
await evm.advanceTimeAndBlock(14*86400)
await game.withdrawLiquidity()

{
  hash: '0x4a64636b61e5be704e793560f11d754505e7a4cbc7ef69e8e1302512ba63a818',
  type: 2,
  accessList: [],
  blockHash: '0xe06a5cdc92a72941fce38af1db9eaebb3063980babcba29c32f0ac1646ca3965',
  blockNumber: 16000050,
  transactionIndex: 0,
  confirmations: 1,
  from: '0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266',
  gasPrice: BigNumber { _hex: '0x3cca9550', _isBigNumber: true },
  maxPriorityFeePerGas: BigNumber { _hex: '0x3b9aca00', _isBigNumber: true },
  maxFeePerGas: BigNumber { _hex: '0x3dfa60a0', _isBigNumber: true },
  gasLimit: BigNumber { _hex: '0x01bad458', _isBigNumber: true },
  to: '0x3D63c50AD04DD5aE394CAB562b7691DD5de7CF6f',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  nonce: 364,
  data: '0x7ea382c1',
  r: '0xc8d5661d06ca7c1573672eca2afaa9e052d5a8dcb8518f19bb7dc8d2553e4023',
  s: '0x06e95d2090880d19b332f0873bd7dcc700e3d483c74161c8aed382b8e65e2180',
  v: 1,
  creates: null,
  chainId: 31337,
  wait: [Function (anonymous)]
}


In [46]:
await game.withdrawPrize()

{
  hash: '0x13c8dbf42250d60cbba80171796bee55dbde02802e4c316afd54c9a6ad8e8781',
  type: 2,
  accessList: [],
  blockHash: '0x4ef50dc4ebb6c7786409c8ef48ed67a9c171077100c4521755b8df400fe4efa2',
  blockNumber: 16000051,
  transactionIndex: 0,
  confirmations: 1,
  from: '0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266',
  gasPrice: BigNumber { _hex: '0x3ca4ca81', _isBigNumber: true },
  maxPriorityFeePerGas: BigNumber { _hex: '0x3b9aca00', _isBigNumber: true },
  maxFeePerGas: BigNumber { _hex: '0x3daecb02', _isBigNumber: true },
  gasLimit: BigNumber { _hex: '0x01bad458', _isBigNumber: true },
  to: '0x3D63c50AD04DD5aE394CAB562b7691DD5de7CF6f',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  nonce: 365,
  data: '0x48d37a58',
  r: '0xb8d27294c1e2b5676387f1491ef3d45f293d31604d0499dd644f483dd7d04788',
  s: '0x043ee089cbedeab2489d00c02ab5481635fb80d95cfa2a89f350e72817c53d07',
  v: 1,
  creates: null,
  chainId: 31337,
  wait: [Function (anonymous)]
}


In [66]:
1e9/0.05e9

20


In [68]:
(1e9**3) / (0.05e9**3)

8000


unknown msg_type: comm_open
unknown msg_type: comm_msg


In [63]:
0.5e9**3/1e18

125000000
